<a href="https://colab.research.google.com/github/fatenchtioui/AD/blob/main/Syst%C3%A8me_de_recommandation_de_contenu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>




# Introduction
Pour créer un système de recommandation de contenu basé sur les préférences de l'utilisateur en utilisant PySpark avec le MovieLens Dataset, suivez les étapes ci-dessous. Le MovieLens Dataset est généralement utilisé pour les recommandations de films.





# Étapes principales :

## Importez les bibliothèques nécessaires :

In [ ]:
!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=c622a1d8de7cf7efcfca6a142d66501c29dfefcf1125c5b764b983602c164a14
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS



## Créez une session Spark :

In [ ]:
spark = SparkSession.builder.appName("MovieRecommendation").getOrCreate()


## Chargez les données MovieLens :

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
movies = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/P_Spark/movie.csv", header=True, inferSchema=True)
ratings = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/P_Spark/rating.csv", header=True, inferSchema=True)


## Explorez les données

In [ ]:
movies.show()
ratings.show()


+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

## Divisez les données en ensembles d'entraînement et de test :

In [ ]:
(training, test) = ratings.randomSplit([0.8, 0.2])


## Configurez le modèle ALS (Alternating Least Squares) :

In [ ]:

als = ALS(
    maxIter=5,
    regParam=0.01,
    userCol="userId",  # Assurez-vous que c'est le bon nom de colonne
    itemCol="movieId",
    ratingCol="rating",
    coldStartStrategy="drop"
)


## Entraînez le modèle sur l'ensemble d'entraînement :

In [ ]:

print(training.columns)


['userId', 'movieId', 'rating', 'timestamp']


In [ ]:
model = als.fit(training)


## Effectuez des prédictions sur l'ensemble de test :

In [ ]:
predictions = model.transform(test)


## Évaluez les performances du modèle

In [ ]:
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="prediction"
)
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) = " + str(rmse))


Root Mean Squared Error (RMSE) = 0.8163080191337452


## Générez des recommandations pour un utilisateur donné

In [ ]:
user_id = 1  # Remplacez par l'ID d'un utilisateur réel
user_recommendations = model.recommendForUserSubset(ratings.where(ratings.userId == user_id), 5)


## Affichez les recommandations

In [ ]:
user_recommendations.show()


+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|     1|[{92083, 11.92922...|
+------+--------------------+



In [ ]:
spark.stop()
